# seq2seq

In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from transformers import get_constant_schedule

In [4]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat, my_reset, get_suitable_for_parquet
from seq2seq_modules.preprocessing import TargetPandasPreprocessor
from seq2seq_modules.models import LSTMModel
from seq2seq_modules.weight_initialization import weights_init_uniform_rule
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import AGE_METRIC

## Read and process

In [5]:
LOCAL_DATA_PATH = './data/'
SPLIT_SEED = 42

In [6]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "domain"
    "capital_marker"
]

continous_features = [
    "request_cnt",
    "price",
    "timestamp",
    "relative_date"
    "geo_lat",
    "geo_lon",
    "population",
    "timezone"
]

In [7]:
df = pandas_reduce_mem_usage(
    pd.read_parquet("seq2seq_data/stages/stage_4.parquet.gzip")
)
df.head()

Memory usage of dataframe is 21327.81 MB


100%|███████████████████████████████████████████| 25/25 [00:09<00:00,  2.70it/s]

Memory usage after optimization is: 20073.23 MB
Decreased by 5.9%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,day_of_week,holiday,capital_marker,timezone,geo_lat,geo_lon,population,domain,timestamp,relative_date
0,21,409,1,589,5788,2,1,20368.0,2,1,...,2,5,2.0,3,45.040161,38.975964,744933,761,165525120.0,3888.0
1,21,409,1,589,12900,2,1,20368.0,2,1,...,6,5,2.0,3,45.040161,38.975964,744933,549,165559680.0,432.0
2,21,409,1,589,17626,2,1,20368.0,0,1,...,6,1,2.0,3,45.040161,38.975964,744933,712,165499200.0,6264.0
3,21,409,1,589,59366,2,1,20368.0,0,1,...,0,5,2.0,3,45.040161,38.975964,744933,712,165265920.0,29592.0
4,21,409,1,589,59366,2,1,20368.0,0,1,...,0,5,2.0,3,45.040161,38.975964,744933,712,165386880.0,17496.0


In [8]:
target = pd.read_parquet(
             f'{LOCAL_DATA_PATH}public_train.pqt', columns=["user_id", "is_male"]
         ).dropna()
target = target[target["is_male"] != "NA"]
target["is_male"] = target["is_male"].astype(np.int32)
target = pandas_reduce_mem_usage(target)

target.head()

Memory usage of dataframe is 5.04 MB


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 2442.81it/s]

Memory usage after optimization is: 3.28 MB
Decreased by 35.0%


,user_id,is_male
350459,350459,1
188276,188276,1
99002,99002,0
155506,155506,0
213873,213873,0


In [9]:
df = df.merge(target, how="left", on="user_id").dropna(subset=["is_male"])
df["is_male"] = df["is_male"].astype(np.int32)
df = pandas_reduce_mem_usage(df)
df.head()

Memory usage of dataframe is 13662.12 MB


100%|███████████████████████████████████████████| 26/26 [00:05<00:00,  4.57it/s]

Memory usage after optimization is: 13059.38 MB
Decreased by 4.4%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,holiday,capital_marker,timezone,geo_lat,geo_lon,population,domain,timestamp,relative_date,is_male
2272,60,732,27,211,5790,2,0,74259.0,3,1,...,5,0.0,3,59.939133,30.315901,4848742,761,163261440.0,10584.0,1
2273,60,732,27,211,65865,2,0,74259.0,0,3,...,5,0.0,3,59.939133,30.315901,4848742,549,162950400.0,41256.0,1
2274,60,732,27,211,111474,2,0,74259.0,0,3,...,5,0.0,3,59.939133,30.315901,4848742,761,162941760.0,42120.0,1
2275,14,311,27,211,111474,2,0,74259.0,1,2,...,5,2.0,2,54.707470,20.507324,431491,761,163114560.0,24624.0,1
2276,60,732,27,211,125409,2,0,74259.0,0,1,...,5,0.0,3,59.939133,30.315901,4848742,549,163028160.0,33480.0,1


In [10]:
df.to_parquet("seq2seq_data/version_3_is_male.parquet.gzip",
              compression='gzip')

## Make torch Dataset

In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import get_constant_schedule, get_cosine_schedule_with_warmup

In [3]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat
from seq2seq_modules.preprocessing import TargetPandasPreprocessor
from seq2seq_modules.data import TargetDataset
from seq2seq_modules.models import LSTMModel, StarterBERTModel, AttentionPoolingBERTModel
from seq2seq_modules.weight_initialization import weights_init_uniform_rule, weights_init_xavier
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import GENDER_METRIC
from seq2seq_modules.utils import fix_random_state

In [4]:
def my_reset(*varnames):
    """
    varnames are what you want to keep
    """
    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

In [5]:
LOCAL_DATA_PATH = './data/'
SEED = 42
fix_random_state(SEED)

In [6]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "domain",
    "capital_marker",
]

continous_features = [
    "request_cnt",
    "price",
    "timestamp",
    "relative_date",
    "geo_lat",
    "geo_lon",
    "population",
    "timezone",
]

In [7]:
df = pandas_reduce_mem_usage(
    pd.read_parquet("seq2seq_data/version_3_is_male.parquet.gzip")
)
df.head()

Memory usage of dataframe is 13059.38 MB


100%|███████████████████████████████████████████| 26/26 [00:06<00:00,  4.06it/s]

Memory usage after optimization is: 13059.38 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,holiday,capital_marker,timezone,geo_lat,geo_lon,population,domain,timestamp,relative_date,is_male
2272,60,732,27,211,5790,2,0,74259.0,3,1,...,5,0.0,3,59.939133,30.315901,4848742,761,163261440.0,10584.0,1
2273,60,732,27,211,65865,2,0,74259.0,0,3,...,5,0.0,3,59.939133,30.315901,4848742,549,162950400.0,41256.0,1
2274,60,732,27,211,111474,2,0,74259.0,0,3,...,5,0.0,3,59.939133,30.315901,4848742,761,162941760.0,42120.0,1
2275,14,311,27,211,111474,2,0,74259.0,1,2,...,5,2.0,2,54.707470,20.507324,431491,761,163114560.0,24624.0,1
2276,60,732,27,211,125409,2,0,74259.0,0,1,...,5,0.0,3,59.939133,30.315901,4848742,549,163028160.0,33480.0,1


In [8]:
gc.collect()

0

In [9]:
dataset = TargetDataset(
         df,
         agg_column="user_id", 
         time_column="timestamp",
         target_column="is_male",
         cat_features=cat_features,
         cont_features=continous_features,
         max_len=1024,
         padding_side="left",
)
dataset

  0%|          | 0/210673077 [00:00<?, ?it/s]

In [10]:
df.head()

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,holiday,capital_marker,timezone,geo_lat,geo_lon,population,domain,timestamp,relative_date,is_male
2272,60,732,27,211,5790,2,0,74259.0,3,1,...,5,0.0,3,59.939133,30.315901,4848742,761,163261440.0,10584.0,1
2273,60,732,27,211,65865,2,0,74259.0,0,3,...,5,0.0,3,59.939133,30.315901,4848742,549,162950400.0,41256.0,1
2274,60,732,27,211,111474,2,0,74259.0,0,3,...,5,0.0,3,59.939133,30.315901,4848742,761,162941760.0,42120.0,1
2275,14,311,27,211,111474,2,0,74259.0,1,2,...,5,2.0,2,54.707470,20.507324,431491,761,163114560.0,24624.0,1
2276,60,732,27,211,125409,2,0,74259.0,0,1,...,5,0.0,3,59.939133,30.315901,4848742,549,163028160.0,33480.0,1


## Feed to the model

In [11]:
cat_feature_indexes = []
cont_feature_indexes = []
vocab_sizes = {}

for i in tqdm(range(len(cat_features))):
    cat_feature_indexes.append(i)
    vocab_sizes[i] = int(df[cat_features[i]].max() + 1)

for i in tqdm(range(len(continous_features))):
    cont_feature_indexes.append(i)

100%|█████████████████████████████████████████| 8/8 [00:00<00:00, 319566.02it/s]


In [12]:
targets = torch.cat([el[3].unsqueeze(0) for el in tqdm(dataset)], dim=0)
targets

100%|████████████████████████████████| 264325/264325 [00:15<00:00, 17056.77it/s]


tensor([0, 0, 0,  ..., 0, 0, 1])

In [13]:
# model = AttentionPoolingBERTModel(
#         cat_feature_indexes=cat_feature_indexes,
#         vocab_sizes=vocab_sizes,
#         cont_feature_indexes=cont_feature_indexes,
#         encoder_hidden_dim=16,
#         hidden_dim=256,
#         dim_feedforward=512,
#         output_dim=7,
#         pe_type="trainable",
#         use_mask=False,
#         max_len=1024,
#         use_key_padding_mask=True,
# )

model = StarterBERTModel(
        cat_feature_indexes=cat_feature_indexes,
        vocab_sizes=vocab_sizes,
        cont_feature_indexes=cont_feature_indexes,
        encoder_hidden_dim=16,
        hidden_dim=256,
        dim_feedforward=512,
        output_dim=2,
        pe_type="trainable",
        use_mask=False,
        max_len=1024,
        use_key_padding_mask=False,
        starter="randn",
        shared=True
)

# model = LSTMModel(
#         cat_feature_indexes=cat_feature_indexes,
#         vocab_sizes=vocab_sizes,
#         cont_feature_indexes=cont_feature_indexes,
#         encoder_hidden_dim=16,
#         hidden_dim=256,
#         output_dim=2,
# )


weights_init_xavier(model)

loss = nn.CrossEntropyLoss(
    weight=torch.tensor(
        compute_class_weight(
            class_weight="balanced",
            classes=sorted(df["is_male"].unique()),
            y=targets.flatten().numpy()
        )
    )
)

metric = GENDER_METRIC

In [15]:
cross_validation(
        project_name="ХУЙ", 
        model=model, 
        dataset=dataset, 
        loss_function=loss, 
        metric_func=metric, 
        optimizer=torch.optim.AdamW, 
        get_scheduler=get_cosine_schedule_with_warmup, 
        strat_array=targets.numpy(), 
        device='cuda', 
        random_state=SEED, 
        shuffle=True, 
        dataloader_shuffle=False, 
        n_folds=5, 
        epochs=7, 
        lr=1e-3, 
        weight_decay=1e-4,
        start_fold=0, 
        batch_size=64,
)

FOLD 0
--------------------------------


  0%|          | 0/3305 [00:00<?, ?it/s]

  0%|          | 0/827 [00:00<?, ?it/s]

EPOCH: 0
{'Gender GINI': 0.09898355270401904, 'loss': 0.6921990615165388}
{'Gender GINI': 0.23257183816157068, 'loss': 0.6768497438105905}


  0%|          | 0/3305 [00:00<?, ?it/s]

...

  0%|          | 0/3305 [00:00<?, ?it/s]

  0%|          | 0/827 [00:00<?, ?it/s]

EPOCH: 6
{'Gender GINI': 0.30867829024023674, 'loss': 0.6376548226244728}
{'Gender GINI': 0.34208887745768357, 'loss': 0.6388367937740074}
FOLD 1
--------------------------------


  0%|          | 0/3305 [00:00<?, ?it/s]

KeyboardInterrupt: 